Carregar o dataset 

In [12]:
import numpy as np
import matplotlib.pyplot as plt
from pgmpy.estimators import BayesianEstimator
from pgmpy.inference import VariableElimination
from sklearn.model_selection import train_test_split
from pgmpy.factors.discrete import TabularCPD
from sklearn.model_selection import train_test_split
import pandas as pd

# Carregamento do dataset
dataset = pd.read_csv(filepath_or_buffer="/home/wyctor/PROJETOS/pad_ufes_20_eda/data/metadata.csv", sep=",")
dataset



,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


In [13]:

dataset.columns

Index(['patient_id', 'lesion_id', 'smoke', 'drink', 'background_father',
       'background_mother', 'age', 'pesticide', 'gender',
       'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'fitspatrick', 'region', 'diameter_1',
       'diameter_2', 'diagnostic', 'itch', 'grew', 'hurt', 'changed', 'bleed',
       'elevation', 'img_id', 'biopsed'],
      dtype='object')

In [14]:

dataset_filtered=dataset.drop(columns=['lesion_id', 'patient_id', 'img_id'])
dataset_filtered=dataset_filtered.dropna()
dataset_filtered

,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,...,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,biopsed
1,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,...,6.0,5.0,BCC,True,True,False,True,True,True,True
4,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,...,5.0,5.0,BCC,True,True,False,False,True,True,True
6,False,True,GERMANY,ITALY,52,False,FEMALE,False,True,True,...,15.0,10.0,BCC,False,True,False,True,True,True,True
7,False,False,POMERANIA,POMERANIA,74,True,FEMALE,False,False,False,...,15.0,10.0,BCC,True,True,True,False,True,True,True
9,False,True,GERMANY,GERMANY,58,True,FEMALE,True,True,True,...,9.0,7.0,ACK,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2289,True,False,POMERANIA,POMERANIA,66,False,MALE,True,False,True,...,20.0,15.0,ACK,True,False,True,False,True,True,True
2291,False,False,POMERANIA,POMERANIA,41,True,MALE,False,False,False,...,9.0,5.0,BCC,True,UNK,True,UNK,True,True,True
2292,False,False,POMERANIA,POMERANIA,43,True,FEMALE,True,True,False,...,9.0,5.0,SCC,True,UNK,False,UNK,True,True,True
2294,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,...,13.0,12.0,BCC,True,True,False,True,False,False,True


In [15]:
X_train, X_test, y_train, y_test =  train_test_split(dataset_filtered.drop(columns=['diagnostic']), dataset_filtered['diagnostic'])

In [22]:
import pgmpy
from pgmpy.models import BayesianModel, BayesianNetwork
from pgmpy.estimators import BayesianEstimator, ParameterEstimator

dag = [
    ('diagnostic', 'grew'),
    ('diagnostic', 'itch'),
    ('diagnostic', 'bleed'),
    ('diagnostic', 'elevation'),
    ('diagnostic', 'hurt'),
    ('diagnostic', 'grew'),
    ('diagnostic', 'changed'),
    # ('bleed', 'diagnostic'),
    ('gender', 'diagnostic'),
    ('region', 'diagnostic'),
    ('skin_cancer_history', 'diagnostic'),
    ('pesticide', 'diagnostic'),
    ('cancer_history', 'diagnostic'),
    ('drink', 'diagnostic'),
    ('smoke', 'diagnostic'),
    ('has_sewage_system', 'diagnostic'),
    ('has_piped_water', 'diagnostic'),
]

model = BayesianNetwork(dag)
graphviz = model.to_graphviz()
graphviz.draw("./results/dag.png", prog="dot")

In [23]:
X_train['diagnostic'] = y_train
model.fit(X_train, estimator=BayesianEstimator)

In [24]:
infer = VariableElimination(model)

In [26]:
print(infer.query(['diagnostic'], evidence={'bleed': 1}))

KeyError: 1